In [2]:
import pandas as pd
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
from kobert_transformers import get_kobert_model
from kobert_transformers.tokenization_kobert import KoBertTokenizer
import numpy as np
import time
from matplotlib import rcParams
# 한글 폰트 설정
rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
rcParams['axes.unicode_minus'] = False     # 음수 기호 문제 해결

In [3]:
df_bigkinds = pd.read_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후.xlsx', converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [4]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,URL,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사
0,02100101.20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...
1,02100101.20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...
2,02100101.20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...
3,02100101.20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...
4,02100311.20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,http://www.sedaily.com/NewsView/1OKUY3ELL1,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,02100601.20240826182228001,2024-08-26,한국경제,오현우,수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다,사회>의료_건강,IT_과학>과학,NaN,NaN,NaN,...,https://www.hankyung.com/article/2024082667731,NaN,"수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술,이송,재활,로봇,의료대,걱...",수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 하는 로봇 의료 대란 걱정 덜었다 수술 이송 재활 로봇 의료 대...,수술 이송 재활 다 는 로봇 의료대란 걱정 덜 었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 걱정 고령 필연 의...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 대 걱정 고령 필연...
6056,02100601.20240826161718002,2024-08-26,한국경제,이해성,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭",사회>사회일반,IT_과학>과학,NaN,NaN,NaN,...,https://www.hankyung.com/article/202408266335i,NaN,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭 삼성전자,기흥,피폭,작업자...",삼성전자 기흥 피폭 작업자

In [5]:
# KoBERT 모델 및 토크나이저 로드
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
model = get_kobert_model()

In [6]:
# 감정 분석을 위한 회귀 레이어 추가
class KoBERTSentimentRegressor(nn.Module):
    def __init__(self, base_model, output_size=1):  # output_size를 1로 설정
        super(KoBERTSentimentRegressor, self).__init__()
        self.base_model = base_model
        self.regressor = nn.Linear(base_model.config.hidden_size, output_size)  # 연속적인 출력값을 위한 선형 레이어
    
    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰의 출력
        return self.regressor(pooled_output)

# 감정 분석 모델 생성
regressor = KoBERTSentimentRegressor(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regressor.to(device)

KoBERTSentimentRegressor(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
# 데이터셋 클래스 정의
class SentimentDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

# 데이터셋 준비
MAX_LEN = 64  # 최대 토큰 길이 설정
dataset = SentimentDataset(df_bigkinds['제목_키워드_전처리후_Komoran'].values, tokenizer, MAX_LEN)
data_loader = DataLoader(dataset, batch_size=16)

In [8]:
# 감정 분석 예측 함수
def predict_sentiments(model, data_loader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            sentiment_scores = model(input_ids=input_ids, attention_mask=attention_mask)
            scores = sentiment_scores.cpu().numpy().flatten()
            # -1 ~ 1 범위로 클리핑
            scores = np.clip(scores, -1, 1)
            predictions.extend(scores)
    
    return predictions

# 감정 분석 수행
s = time.time()
predicted_scores = predict_sentiments(regressor, data_loader, device)
print("Prediction Time: ", time.time()-s)

C:\Users\dybae\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Prediction Time:  46.298773765563965


In [9]:
# 뉴스 식별자 컬럼에서 '.' 이후의 값만 조회
df_bigkinds['뉴스 식별자'] = df_bigkinds['뉴스 식별자'].str.split('.').str[1]

In [10]:
# 데이터프레임에 결과 추가
df_bigkinds['Bert_Sentiment'] = predicted_scores
# 소수점 1자리로 반올림
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].round(1)
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,Bert_Sentiment
0,20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...,0.1
1,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.1
2,20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...,0.0
3,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
4,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,20240826182228001,2024-08-26,한국경제,오현우,수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다,사회>의료_건강,IT_과학>과학,NaN,NaN,NaN,...,NaN,"수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술,이송,재활,로봇,의료대,걱...",수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 하는 로봇 의료 대란 걱정 덜었다 수술 이송 재활 로봇 의료 대...,수술 이송 재활 다 는 로봇 의료대란 걱정 덜 었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 걱정 고령 필연 의...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 대 걱정 고령 필연...,0.1
6056,20240826161718002,2024-08-26,한국경제,이해성,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭",사회>사회일반,IT_과학>과학,NaN,NaN,NaN,...,NaN,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭 삼성전자,기흥,피폭,작업자...",삼성전자 기흥 피폭 작업자 손에 기준치의 188배 피폭 삼성전자 기흥 피폭 작업자 ...,삼성 기흥 피폭 작업 손 기준 치의 188 배 피폭 삼성 기흥 피폭 작업 기 준치 ...,삼성전자 기흥 피폭 작업자 손 기준치 188배 피폭 삼성전자 기흥 피폭 작업자 기준...,삼성전자 기흥 피폭 작업자 손 기준치 188 배 피폭 삼성전자 기흥 피폭 작업자 기...,삼성전자 기흥 피폭 작업자 손 기준 치 188 배 피폭 삼성전자 기흥 피폭 작업자 ...,삼성전자 기흥 피폭 작업자 기준 188 피폭 삼성전자 기흥 피폭 작업자 기준 피폭 ...,삼성전자 기흥 피폭 작업자 손 기준 배 피폭 삼성전자 기흥 피폭 작업자 기준 피폭 ...,0.1
6057,20240827181505005,20

In [11]:
df_bigkinds.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', '제목_키워드', '제목_키워드_전처리후',
       '제목_키워드_전처리후_Okt', '제목_키워드_전처리후_Hannanum', '제목_키워드_전처리후_Kkma',
       '제목_키워드_전처리후_Komoran', '제목_키워드_전처리후_kiwi', '제목_키워드_전처리후_Komoran_명사',
       'Bert_Sentiment'],
      dtype='object')

In [12]:
df_bigkinds.to_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Komoran_KoBERT.xlsx', index=False)

In [13]:
df_bigkinds = df_bigkinds.groupby('일자', as_index=False)['Bert_Sentiment'].mean()
# 소수점 1자리로 반올림
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].round(1)
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,일자,Bert_Sentiment
0,2017-09-01,0.1
1,2017-09-04,0.1
2,2017-09-05,0.1
3,2017-09-06,0.1
4,2017-09-07,0.1
...,...,...
1719,2024-08-23,0.2
1720,2024-08-25,0.1
1721,2024-08-26,0.1
1722,2024-08-27,0.1


In [14]:
# 감정 분석 결과 저장(일자별)
df_bigkinds.to_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Komoran_KoBERT_daily.xlsx', index=False)

In [15]:
#KoBERT의 경우 다시 평가하는 경우 시간 소요 및 데이터가 변경되어 기존에 평가 데이터를 필터링하여 정상 데이터만 추출
import pandas as pd
# EXCEL파일 읽기
file_path = './Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Komoran_KoBERT.xlsx'
df_bigkinds = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

df_bigkinds = df_bigkinds[df_bigkinds['분석제외 여부'].isnull()]

df_bigkinds

# 그룹핑할 컬럼 리스트
group_columns = ['제목_키워드_전처리후_Komoran']

# 각 그룹에서 뉴스 식별자 컬럼의 최소값을 유지
df_bigkinds2 = df_bigkinds.groupby(group_columns, as_index=False).agg({'뉴스 식별자': 'min'})

# 필요한 컬럼만 선택하여 두 데이터프레임 병합
big_kinds_sentiment = pd.merge(
    df_bigkinds[['제목_키워드_전처리후_Komoran', '뉴스 식별자','일자','Bert_Sentiment']],
    df_bigkinds2[['제목_키워드_전처리후_Komoran', '뉴스 식별자']],
    on=['제목_키워드_전처리후_Komoran', '뉴스 식별자'],
    how='inner'
)

big_kinds_sentiment = big_kinds_sentiment[['뉴스 식별자', '일자', '제목_키워드_전처리후_Komoran', 'Bert_Sentiment']]
big_kinds_sentiment

big_kinds_sentiment.to_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Komoran_KoBERT_filter.xlsx', index=False)

big_kinds_sentiment_avg = big_kinds_sentiment.groupby('일자', as_index=False)['Bert_Sentiment'].mean()
# 소수점 1자리로 반올림
big_kinds_sentiment_avg['Bert_Sentiment'] = big_kinds_sentiment_avg['Bert_Sentiment'].round(1)
big_kinds_sentiment_avg['Bert_Sentiment'] = big_kinds_sentiment_avg['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
big_kinds_sentiment_avg

# 감정 분석 결과 저장(일자별)
big_kinds_sentiment_avg.to_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Komoran_KoBERT_daily_filter.xlsx', index=False)

In [16]:
big_kinds_sentiment_avg

,일자,Bert_Sentiment
0,2017-09-01,0.1
1,2017-09-04,0.1
2,2017-09-05,0.1
3,2017-09-06,0.0
4,2017-09-07,0.1
...,...,...
1646,2024-08-22,0.0
1647,2024-08-23,0.2
1648,2024-08-25,0.1
1649,2024-08-26,0.1


In [17]:
big_kinds_sentiment

,뉴스 식별자,일자,제목_키워드_전처리후_Komoran,Bert_Sentiment
0,20170901163910001,2017-09-01,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,0.1
1,20170901174354001,2017-09-01,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,0.1
2,20170901181209001,2017-09-01,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,0.0
3,20170901141935001,2017-09-01,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,0.0
4,20170901153936001,2017-09-01,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,0.0
...,...,...,...,...
5475,20240823175054001,2024-08-23,현대차 쏘 올리 ㄴ 산업 계 재 고용 바람 현대차 바람 산업 정년 생산 근로자 재 ...,0.2
5476,20240825142511001,2024-08-25,부산 국제 마케팅 광고 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고 그랑프리 ...,0.1
5477,20240826182228001,2024-08-26,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,0.1
5478,20240826161718002,2024-08-26,삼성전자 기흥 피폭 작업자 손 기준 치 188 배 피폭 삼성전자 기흥 피폭 작업자 ...,0.1
